In [3]:
import sys
sys.path.append('../')

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
%reload_ext autoreload

In [64]:
from scripts.detect_voids import get_neightbours
from packages.yolo_predict import YOLO_Pred
import pandas as pd
import cv2

In [13]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [95]:
df_predictions = yolo.predictions('/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [113]:
df_predictions

,Center_X,Center_Y,Width,Height
572,221.571442,307.081207,23.514479,68.048775
1059,210.405991,233.077942,24.668375,70.583984
581,314.498230,306.780518,16.290375,60.977318
585,343.513489,306.846436,16.417421,60.421963
1203,148.945190,518.730042,27.151506,27.399454
...,...,...,...,...
1145,119.181030,465.013733,24.931517,14.736507
712,446.199127,483.145142,42.025547,15.644592
310,427.420837,317.954681,10.442470,28.763885
600,167.968338,316.750610,21.088881,58.980614


In [86]:
dict_of_neightbours = get_neightbours(img_path='/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [115]:
dict_of_neightbours[0]['572']

[571, 564, 273, 277, 275, 599, 600]

In [88]:
len(df_predictions)

161

In [ ]:
for index in df_predictions.index:
    print(index)
    for list_ in dict_of_neightbours[0][str(index)]:
        print(list_)

In [89]:
len(dict_of_neightbours[0])

152

In [104]:
dict_of_neightbours[0]['43']

[632]

In [63]:
dict_of_neightbours[0]['572']

[571, 564, 273, 277, 275, 599, 600]

In [82]:
def bb_intersection_over_union(df_predictions: pd.DataFrame, index_key:int, index_neightbour:int) -> float:

    a = index_key
    xA1 = df_predictions.loc[a][0] - df_predictions.loc[a][2]/2
    yA1 = df_predictions.loc[a][1] + df_predictions.loc[a][3]/2
    xA2 = df_predictions.loc[a][0] + df_predictions.loc[a][2]/2
    yA2 = df_predictions.loc[a][1] - df_predictions.loc[a][3]/2
    boxA = [xA1, yA1, xA2, yA2 ]

    b = index_neightbour # indices vecinos detectados que estan en el diccionario
    xB1 = df_predictions.loc[b][0] - df_predictions.loc[b][2]/2
    yB1 = df_predictions.loc[b][1] + df_predictions.loc[b][3]/2
    xB2 = df_predictions.loc[b][0] + df_predictions.loc[b][2]/2
    yB2 = df_predictions.loc[b][1] - df_predictions.loc[b][3]/2
    boxB = [xB1, yB1,xB2, yB2 ]
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])


    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
# para left w = -w y h = 0
# para right w = +w y h = 0
# para top w = 0 y h = + h

In [90]:
df_predictions_test = df_predictions.copy()

In [94]:
df_predictions_test.loc[1260]

Center_X             194.691193
Center_Y             139.134720
Width                 22.782619
Height                90.207382
Height_over_image    184.238411
Name: 1260, dtype: float64

In [91]:
df_predictions_test['Height_over_image'] =  df_predictions_test.Center_Y + (df_predictions_test.Height/2)

In [92]:
df_predictions_test[df_predictions_test['Height_over_image'] == df_predictions_test['Height_over_image'].max()]

,Center_X,Center_Y,Width,Height,Height_over_image
1250,18.964607,563.583618,26.012243,46.716412,586.941824


In [60]:
df_predictions.loc[572][2]

23.51447868347168

In [65]:
image = cv2.imread(filename='/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [72]:
image.shape[0:2]

(3264, 2448)

In [132]:
# para el diccionario de neightbours 

for index_a, index_b in dict_of_neightbours[0].items():
    print(f'This bb {index_a} has neightbours')
    h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
    w_index_a = df_predictions.loc[int(index_a)][2]
    xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - w_index_a
    yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
    xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - w_index_a
    yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2
    boxA = [xA1, yA1, xA2, yA2]
    if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
        for item in index_b:
            trigger = False 
            print(f'Neightbour bound box: {item}')
            xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
            yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
            xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
            yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
            boxB = [xB1, yB1,xB2, yB2]

            xA = max(boxA[0], boxB[0])
            yA = max(boxA[1], boxB[1])
            xB = min(boxA[2], boxB[2])
            yB = min(boxA[3], boxB[3])

            interArea = (xB - xA) * (yB - yA)

            boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
            boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

            iou = interArea / float(boxAArea + boxBArea - interArea)
            if iou <= 0.05:
                trigger = True
            else:
                trigger = False
        if trigger:
            print(f'A la izquierda del bb: {index_a} hay un espacio VACIO {boxA}')

This bb 572 has neightbours
Neightbour bound box: 571
Neightbour bound box: 564
Neightbour bound box: 273
Neightbour bound box: 277
Neightbour bound box: 275
Neightbour bound box: 599
Neightbour bound box: 600
A la izquierda del bb: 572 hay un espacio VACIO [186.2997236251831, 273.0568199157715, 209.81420230865479, 341.10559463500977]
This bb 1059 has neightbours
Neightbour bound box: 1054
Neightbour bound box: 825
Neightbour bound box: 1055
A la izquierda del bb: 1059 hay un espacio VACIO [173.40342807769775, 197.78594970703125, 198.07180309295654, 268.36993408203125]
This bb 581 has neightbours
Neightbour bound box: 576
Neightbour bound box: 577
Neightbour bound box: 580
Neightbour bound box: 579
Neightbour bound box: 317
Neightbour bound box: 20
A la izquierda del bb: 581 hay un espacio VACIO [290.0626678466797, 276.2918586730957, 306.35304260253906, 337.2691764831543]
This bb 585 has neightbours
Neightbour bound box: 581
Neightbour bound box: 583
Neightbour bound box: 580
Neightbou

In [130]:
df_predictions.loc[1059]

Center_X    210.405991
Center_Y    233.077942
Width        24.668375
Height       70.583984
Name: 1059, dtype: float64

In [129]:
df_predictions.loc[1059][2]

24.66837501525879

In [123]:
xA1 = df_predictions.loc[1059][0] - df_predictions.loc[1059][2]/2 - w_index_a
yA1 = df_predictions.loc[1059][1] - df_predictions.loc[1059][3]/2
xA2 = df_predictions.loc[1059][0] + df_predictions.loc[1059][2]/2 - w_index_a
yA2 = df_predictions.loc[1059][1] + df_predictions.loc[1059][3]/2

In [125]:
xA1, yA1, xA2, yA2

(184.86670780181885,
 268.36993408203125,
 209.53508281707764,
 197.78594970703125)

In [134]:
dict_of_neightbours[0]['572']

[571, 564, 273, 277, 275, 599, 600]

In [135]:
df_predictions.loc[571]

Center_X    199.205215
Center_Y    306.348145
Width        22.828110
Height       64.707008
Name: 571, dtype: float64

In [ ]:
h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
w_index_a = df_predictions.loc[int(index_a)][2]
a = index_a
xA1 = df_predictions.loc[a][0]-df_predictions.loc[a][2]/2
yA1 = df_predictions.loc[a][1]+df_predictions.loc[a][3]/2
xA2 = df_predictions.loc[a][0]+df_predictions.loc[a][2]/2
yA2 = df_predictions.loc[a][1]-df_predictions.loc[a][3]/2
boxA = [xA1, yA1, xA2, yA2 ]

for 
    b = index_b # indices vecinos detectados que estan en el diccionario
    xB1 = df_predictions.loc[b][0]-df_predictions.loc[b][2]/2
    yB1 = df_predictions.loc[b][1]+df_predictions.loc[b][3]/2
    xB2 = df_predictions.loc[b][0]+df_predictions.loc[b][2]/2
    yB2 = df_predictions.loc[b][1]-df_predictions.loc[b][3]/2
    boxB = [xB1, yB1,xB2, yB2 ]
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])


    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    if iou <= 0.05:
        trigger = True
    else:
        trigger = False
if trigger:
    print(f'A la izquierda del bb: {index_a} hay un espacio VACIO {boxA}')

In [136]:
for item in dict_of_neightbours[0]['216']:
    print(values)

1028
1027
791


In [142]:
index_a = 216
h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
w_index_a = df_predictions.loc[216][2]
xA1 = df_predictions.loc[216][0] - df_predictions.loc[216][2]/2 - w_index_a
yA1 = df_predictions.loc[216][1] - df_predictions.loc[216][3]/2
xA2 = df_predictions.loc[216][0] + df_predictions.loc[216][2]/2 - w_index_a
yA2 = df_predictions.loc[216][1] + df_predictions.loc[216][3]/2
boxA = [xA1, yA1, xA2, yA2]
if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
        for item in dict_of_neightbours[0]['216']:
            trigger = False 
            print(f'Neightbour bound box: {item}')
            xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
            yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
            xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
            yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
            boxB = [xB1, yB1,xB2, yB2]

            xA = max(boxA[0], boxB[0])
            yA = max(boxA[1], boxB[1])
            xB = min(boxA[2], boxB[2])
            yB = min(boxA[3], boxB[3])

            interArea = (xB - xA) * (yB - yA)

            boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
            boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

            iou = interArea / float(boxAArea + boxBArea - interArea)
            #poner una condicion que si ALGUNO de los vecinos tiene IOU > 25%, HAY ALGO y NO HAY espacios vecinos
            if iou >= 0.05:
                print(f'BB vecino {item} con IOU {iou}')
                trigger = True
            else:
                trigger = False
        if trigger:
            print(f'A la izquierda del bb: {index_a} hay un espacio VACIO {boxA}')

Neightbour bound box: 1028
Neightbour bound box: 1027
BB vecino 1027 con IOU 0.02263987022572477
Neightbour bound box: 791
BB vecino 791 con IOU -0.3257936184935919
A la izquierda del bb: 216 hay un espacio VACIO [117.82699966430664, 102.7520866394043, 151.54630661010742, 177.71504592895508]
